In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
import random
import joblib
import os
os.chdir('Resources/')

from custom_tuple_scaler import CustomTupleScaler

In [2]:
df = pd.read_csv('7_CC_Encrypted_Data.csv')

In [3]:
Y = df[['HeartDisease']]
Y

,HeartDisease
0,0
1,1
2,0
3,1
4,0
...,...
4039,1
4040,1
4041,0
4042,0


In [4]:
c1_list = []
c2_list = []

for k in range(11):
    cnt = 0
    c1_sum = 0
    c2_sum = 0
    
    messages = df.iloc[:, k].values

    for i, message in enumerate(messages, 1):
        if message is not None and message == message:
            c1, c2 = map(int, message.strip("()").split(","))
            c1_sum += c1
            c2_sum += c2
            cnt += 1
    
    print(c1_sum//cnt, ",", c2_sum//cnt)

    c1_list.append(c1_sum//cnt)
    c2_list.append(c2_sum//cnt)

print(c1_list)
print(c2_list)

11611711856791845453844363544193004458436652567582 , 8642669844787942563492368268684939284711804276863
11521613315689790599790221322656441619898359351715 , 4034377802708626089760659764802423667325298890365
5340627710438748782301236172298939673737240321334 , 9518105488393556837220105912093619880523995570700
2959799491122696844814186940885321444153216238962 , 8836753688425624510165520513314840242974747212475
2572745932559499053547019353918953850096268849190 , 8474196311723243744418844799390940207929097720980
8991270131106317065828259596356481212681246291421 , 6772656204179509619791305715810025426227694403263
3734041290046056160613914136571085705122900972458 , 10241515065873839029286567858642248780707707120098
11607967323062549316964231219539938367253314058306 , 8434009008481148221040720356012447928285472689883
13647169038252931707909305178839774745227353223390 , 7296217498572403303360981577425449678087323016318
2127171966273882616386704855669834874862200078566 , 8091959164370435025476562

In [5]:
column_names = [
    "Age", "Sex", "ChestPainType", "RestingBloodPressure", "Cholesterol",
    "FastingBloodSugar", "RestingECG", "MaximumHeartRate", "ExerciseAngina",
    "Oldpeak", "ST_Slope", "HeartDisease"
]

In [6]:
imputed_data = pd.DataFrame()

for k in range(11):
    
    messages = df.iloc[:, k].values

    imputed_column = []

    for i, message in enumerate(messages, 1):
        if message is None:
            c1 = c1_list[k]
            c2 = c2_list[k]
        
        if message is not None and message == message:
            c1, c2 = map(int, message.strip("()").split(","))

        imputed_column.append(f"({c1},{c2})")

    imputed_data[column_names[k]] = imputed_column

In [7]:
scaler = joblib.load('13_CC_Scaler.joblib')

In [8]:
df_scaled = scaler.transform(imputed_data)
df_scaled = pd.DataFrame(df_scaled, columns=imputed_data.columns)
# df_scaled.to_csv('scaled_new_data.csv', index=False)

g:\Thesis - Copy\Resources\custom_tuple_scaler.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  c2_values = X.applymap(self._extract_c2)


In [9]:
with open("12_SP_Test_Data_RF_IF.txt", "r") as file:
    test_indices = eval(file.read()) 

In [10]:
file_path = "13_CC_F1_Scores_RF_IF.txt"

with open(file_path, "r") as f:
    lines = f.readlines()

reference_f1_scores = []
for line in lines:
    if "F1-score for fold" in line:
        score = float(line.split(":")[1].strip())
        reference_f1_scores.append(score)

In [11]:
fold_to_test = random.randint(1, 5)

test_indices_fold = test_indices[fold_to_test - 1] 
print(f"Test indices for fold {fold_to_test}: {test_indices_fold}")

X_test = df_scaled.iloc[test_indices_fold]

model_path = f'12_SP_Model_RF_IF_{fold_to_test}.joblib'
model = joblib.load(model_path)

Y_test = Y.iloc[test_indices_fold]
Y_pred = model.predict(X_test)

computed_f1_fold = f1_score(Y_test, Y_pred) * 100

reference_f1_fold = reference_f1_scores[fold_to_test - 1]

print(f"Computed F1-score for fold 1: {computed_f1_fold:.2f}")
print(f"Reference F1-score for fold 1: {reference_f1_fold:.2f}")

if np.isclose(computed_f1_fold, reference_f1_fold, atol=1e-2):
    print("The computed F1-score matches the reference score.")
else:
    print("The computed F1-score does not match the reference score.")

Test indices for fold 4: [1, 2, 3, 4, 6, 11, 13, 22, 24, 25, 32, 33, 40, 51, 55, 57, 60, 66, 70, 74, 76, 78, 81, 85, 87, 90, 107, 117, 118, 121, 122, 124, 133, 136, 140, 142, 144, 147, 158, 159, 164, 165, 166, 170, 177, 178, 179, 183, 184, 185, 189, 190, 196, 206, 209, 210, 216, 222, 227, 229, 240, 243, 246, 250, 258, 261, 267, 272, 274, 279, 284, 287, 310, 316, 319, 328, 329, 331, 334, 351, 353, 354, 361, 363, 371, 386, 396, 404, 416, 417, 418, 427, 438, 441, 448, 449, 450, 453, 461, 464, 465, 473, 482, 485, 486, 487, 490, 491, 495, 499, 502, 521, 532, 534, 536, 538, 541, 542, 543, 550, 551, 555, 569, 573, 580, 585, 597, 598, 602, 608, 609, 617, 621, 647, 652, 657, 666, 668, 669, 670, 671, 674, 675, 676, 677, 683, 684, 690, 693, 694, 702, 706, 707, 725, 726, 727, 729, 731, 739, 748, 751, 757, 758, 765, 779, 787, 788, 799, 801, 807, 836, 859, 865, 868, 874, 883, 884, 885, 887, 900, 908, 911, 917, 925, 931, 934, 947, 949, 952, 953, 955, 974, 982, 987, 988, 990, 995, 996, 1000, 1002, 100